# Sending tesnet bitcoins using only the networking protocol

### You have been sent some unknown amount of testnet bitcoins to your address. 

Send all of it back (minus fees) to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv` using only the networking protocol.

In [4]:
import bloomfilter, merkleblock

from block import Block
from bloomfilter import (
    BloomFilter,
    BIP37_CONSTANT,
)
from ecc import PrivateKey
from helper import (
    bit_field_to_bytes,
    decode_base58,
    hash160,
    hash256,
    little_endian_to_int,
    murmur3,
    run,
    SIGHASH_ALL,
)
from merkleblock import MerkleBlock, MerkleTree
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    SimpleNode
)
from tx import (
    Tx,
    TxIn,
    TxOut,
)

# Exercise 4.1
last_block_hex = '000000000d65610b5af03d73ed67704713c9b734d87cf4b970d39a0416dd80f9'
last_block = bytes.fromhex(last_block_hex)
passphrase = b'Jimmy Song Programming Blockchain'  # FILL THIS IN
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
h160 = decode_base58(addr)
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
filter_size = 30
filter_num_functions = 5
filter_tweak = 90210  # FILL THIS IN
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 5000  # fee in satoshis

# connect to tbtc.programmingblockchain.com in testnet mode, logging True
node = SimpleNode('tbtc.programmingblockchain.com', testnet=True, logging=True)
# create a bloom filter using variables above
bf = BloomFilter(filter_size, filter_num_functions, filter_tweak)
# add the h160 to the bloom filter
bf.add(h160)
# complete the handshake
node.handshake()
# send the 'filterload' command with bf.filterload() as the payload
node.send(b'filterload', bf.filterload())

# create GetHeadersMessage with the last_block as the start_block
getheaders_message = GetHeadersMessage(start_block=last_block)
# send a getheaders message
node.send(getheaders_message.command, getheaders_message.serialize())

# wait for the headers message
headers_envelope = node.wait_for_commands([HeadersMessage.command])
# get the stream from the headers
stream = headers_envelope.stream()
# parse the headers message
headers = HeadersMessage.parse(stream)

# initialize the GetDataMessage
get_data_message = GetDataMessage()
# loop through the blocks in the headers message
for block in headers.blocks:
    # check that the proof of work on the block is valid
    if not block.check_pow():
        raise RuntimeError
    # check that this block's prev_block is the last block
    if last_block is not None and block.prev_block != last_block:
        raise RuntimeError
    # set the last block to the current hash
    last_block = block.hash()
    # add_data(FILTERED_BLOCK_DATA_TYPE, last_block) to get_data_message
    get_data_message.add_data(FILTERED_BLOCK_DATA_TYPE, last_block)
# send the get_data_message
node.send(get_data_message.command, get_data_message.serialize())

# initialize prev_tx to None
prev_tx = None
# while prev_tx is None 
while prev_tx is None:
    # wait for the merkleblock or tx commands
    envelope = node.wait_for_commands([b'merkleblock', b'tx'])
    # initialize the stream from the envelope
    stream = envelope.stream()
    # if we have the merkleblock command
    if envelope.command == b'merkleblock':
        # parse the MerkleBlock
        mb = MerkleBlock.parse(stream)
        # check that the MerkleBlock is valid
        if not mb.is_valid():
            raise RuntimeError
    # else we have the tx command
    else:
        # parse the tx (prev)
        prev = Tx.parse(stream, testnet=True)
        # loop through the enumerated tx outs (enumerate(prev.tx_outs))
        for i, tx_out in enumerate(prev.tx_outs):
            # if our output has the same address as our address (addr) we found it
            if tx_out.script_pubkey.address(testnet=True) == addr:
                # we found our utxo. set prev_tx, prev_index, prev_amount
                prev_tx = prev.hash()
                prev_index = i
                prev_amount = tx_out.amount
                # break
                break
# create tx_in
tx_in = TxIn(prev_tx, prev_index)
# calculate the output amount (prev_amount - fee)
output_amount = prev_amount - fee
# create tx_out
tx_out = TxOut(output_amount, target_script)
# create transaction on testnet
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
# sign the one input we have
tx_obj.sign_input(0, private_key)
# serialize and hex to see what it looks like
print(tx_obj.serialize().hex())
# send this signed transaction on the network
node.send(b'tx', tx_obj.serialize())
# wait a sec so this message goes through to the other node sleep(1) 
sleep(1)
# now ask for this transaction from the other node
# create a GetDataMessage
getdata = GetDataMessage()
# add_data (TX_DATA_TYPE, tx_obj.hash()) to get data message
getdata.add_data(TX_DATA_TYPE, tx_obj.hash())
# send the GetDataMessage
node.send(getdata.command, getdata.serialize())
# now wait for a response
envelope = node.wait_for_commands([b'tx', b'reject'])
# if we have the tx command
if envelope.command == b'tx':
    # parse the tx
    got = Tx.parse(envelope.stream())
    # check that the ids are the same
    if got.id() == tx_obj.id():
        # yes! we got to what we wanted
        print('success!')
        print(tx_obj.id())
else:
    print(envelope.payload)

mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv
sending: version: 7f1101000000000000000000d6daf25b00000000000000000000000000000000000000000000ffff000000008d20000000000000000000000000000000000000ffff000000008d20f5460df5cf9cebfa1b2f70726f6772616d6d696e67626c6f636b636861696e3a302e312f0000000001
receiving: version: 7f1101000d04000000000000d6daf25b00000000000000000000000000000000000000000000ffffc0f19dc0ea0c0d04000000000000000000000000000000000000000000000000d6f90cc62df4edba102f5361746f7368693a302e31372e302f0209160001
sending: verack: 
receiving: verack: 
sending: filterload: 1e000000000448000000000004000000000200000000000000000000000000050000006260010001
sending: getheaders: 7f11010001f980dd16049ad370b9f47cd834b7c913477067ed733df05a0b61650d000000000000000000000000000000000000000000000000000000000000000000000000
receiving: sendheaders: 
receiving: sendcmpct: 000200000000000000
receiving: sendcmpct: 000100000000000000
receiving: ping: 3c326a8ba1b32691
sending: pong: 3c326a8ba1b32691
receiving: addr: 01d6d

sending: getdata: ...
receiving: merkleblock: 00000020f980dd16049ad370b9f47cd834b7c913477067ed733df05a0b61650d00000000f77782adc2f061d14c804a1f555d13105452a39ce7bd871afc571fca350f565c3216e75bffff001da04e6e197200000008bef47547638f77d1890fa382e45706ab43e30fde6428d63cbbcee8cbdba3358894e631abb9e1079ec72a1616a3aa0111c614e65b96a6a4420e2cc6af9e6cc96e8374eb69e45152a0743f50926d8702e3594d33f488b826e772b24b2ae6beee6026c734eb5b3c05683e29fe4f3b58a156e2abe0af2ad69c3e30ba117c701bee8ff10a32e1ac3641c029f02f044bb2364233b77739bbf230b662db118aac19a440fa146e12029eba033575c125d5a8533c3ddf34e3044c7d334d0fcc41c4c186fbd184d300999e65ba67295a8b85ef2eea75177452816858ef7aa59b3a0d40a7ccc0a80bc2b5749751808e378bef0e713173291e207068e29dd716169dfb00abc702bf01
receiving: tx: 0100000002a663815ab2b2ba5f53e442f9a2ea6cc11bbcd98fb1585e48a134bd870dbfbd6a000000006a47304402202151107dc2367cf5a9e2429cde0641c252374501214ce52069fbca1320180aa602201a43b5d4f91e48514c00c01521dc04f02c57f15305adc4eaad01c418f6e7a1180121031dbe3aff7b9ad64e26

010000000194e631abb9e1079ec72a1616a3aa0111c614e65b96a6a4420e2cc6af9e6cc96e000000006a47304402203cc8c56abe1c0dd043afa9eb125dafbebdde2dd4cd7abf0fb1aae0667a22006e02203c95b74d0f0735bbf1b261d36e077515b6939fc088b9d7c1b7030a5e494596330121021cdd761c7eb1c90c0af0a5963e94bf0203176b4662778d32bd6d7ab5d8628b32ffffffff01f8829800000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000
sending: tx: 010000000194e631abb9e1079ec72a1616a3aa0111c614e65b96a6a4420e2cc6af9e6cc96e000000006a47304402203cc8c56abe1c0dd043afa9eb125dafbebdde2dd4cd7abf0fb1aae0667a22006e02203c95b74d0f0735bbf1b261d36e077515b6939fc088b9d7c1b7030a5e494596330121021cdd761c7eb1c90c0af0a5963e94bf0203176b4662778d32bd6d7ab5d8628b32ffffffff01f8829800000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000
sending: getdata: 0101000000fb587d34ba83d588654aafc584af815b3aee643e962ab75f8b3346e5cbf1fca8
receiving: merkleblock: 00000020e98ce8fa31abc4bd31dc9105ecb84aafa9a0d029c0d89ded14787853000000000c71c1a8ce1436d96c1218b7b7fdb0d6